In [1]:
import numpy as np
import pandas as pd 

In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
from torch import Tensor
from torch_geometric.nn import GCNConv
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import glob
import os

In [6]:
def weights():
    weights=[0.0055,0]
    for i in range(2,21):
        weights.append(0.005)
    for i in range(21,31):
        weights.append(0.006)
    for i in range(31,37):
        weights.append(0.0065)
    for i in range(37,85):
        weights.append(0.00675)
    for i in range(85,93):
        weights.append(0.0068)
    weights.append(0.0065)
    weights.append(0.0065)
    for i in range(95,140):
        weights.append(0.01)
    return weights


In [7]:
def replacetensor(tensor,process):
    dup=tensor.clone()
    dup=dup.detach().numpy()
    dup=dup.tolist()
    tensor,indices=torch.sort(tensor)
    i=0
    ind=dup.index(tensor[0])
    dup[ind]=i
    for j in range(1,process):
        ind=dup.index(tensor[j])
        if tensor[j-1]<tensor[j]:
            i+=1
        dup[ind]=i
        dup[ind]=int(dup[ind])
    dup=np.array(dup)
    dup=torch.from_numpy(dup)
    dup=dup.type(torch.LongTensor)
    return dup

In [8]:
def runtimepernode(weight,feature_vector):
    lf=feature_vector.shape
    rv=[]
    for i in range(lf[0]):
        rt=0
        for j in range(lf[1]):
            rt+=weight[j]*feature_vector[i][j]
        rv.append(rt)
    return rv

In [9]:
def addruntime(edges_vector,cumruntime):
    le=len(edges_vector)
    fr=cumruntime
    fr[edges_vector[0][0]]=(cumruntime[edges_vector[0][0]]+cumruntime[edges_vector[0][1]])
    for i in range(1,le):
        if cumruntime[edges_vector[i-1][0]]!=cumruntime[edges_vector[i][0]]:
            fr[edges_vector[i][0]]=(cumruntime[edges_vector[i][0]]+cumruntime[edges_vector[i][1]])
        else:
            fr[edges_vector[i][0]]+=cumruntime[edges_vector[i][1]]
    return fr

In [10]:
def arrangetiles(pred_tensor,size):
    order=[]
    dic={}
    sd={}
    rl=[]
    for i in range(size):
        maxi=pred_tensor[i,:].max()
        for j in range(size):
            if pred_tensor[i][j]==maxi:
                order.append(j)
                break
    for i in order:
        if i in dic:
            dic[i]+=1
        else:
            dic[i]=1
    sorted_keys = sorted(dic, key=dic.get,reverse=True)
    for w in sorted_keys:
        sd[w] = dic[w]
    j=0
    for i in sd:
        j+=1
        if j<6:
            rl.append(i)
        else:
            break
    return rl

In [11]:
t=glob.glob(os.path.join('/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla', "test", "*.npz"))
tile_order=[]
lfile=len(t)
wt=weights()
for i in range(838,lfile):
    d_tiletest=dict(np.load(t[i]))
    t_f=torch.from_numpy(d_tiletest["config_feat"])
    t_e=torch.from_numpy(d_tiletest["edge_index"])
    t_e=torch.transpose(t_e, 0, 1)
    process=len(d_tiletest["config_runtime"])
    model = GCN(24,16,process)
    over_run=runtimepernode(wt,d_tiletest["node_feat"])
    final_run=addruntime(d_tiletest["edge_index"],over_run)
    total_run=sum(final_run)
    runtime=[]
    l=len(d_tiletest["config_runtime"])
    for i in range(l):
        runtime.append(d_tiletest["config_runtime"][i]+sum(d_tiletest["config_feat"][i])/total_run)
    runtime=np.array(runtime)
    runtime=torch.from_numpy(runtime)
    runtime=replacetensor(runtime,process)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    for epoch in range(20):
        pred = model(t_f,t_e)
        loss = F.cross_entropy(pred,runtime)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    order=np.array(arrangetiles(pred,process))
    tile_order.append(order)
    del d_tiletest
    del runtime
    del pred
    del order

In [12]:
to=np.array(tile_order)

In [13]:
to

array([[176, 106,  16, 229, 103],
       [280, 174, 142,  39, 314],
       [ 16,  81,  60,  25,  83],
       [ 82,  33, 132,  25,  97],
       [477,  33, 285,  72,  95],
       [ 25,  10, 109, 208, 139]])

In [14]:
len(to)

6

In [15]:
np.save('myarray.npy', to)

model1=GCN(24,16,266)
model1.load_state_dict(torch.load("/kaggle/input/graphnn/tilerun.pt"))
model1.fc=GCN(24,16,process)

pred1=model1(t_f,t_e)